<bn>

# Sensorimotor experiment

![대체 텍스트](http://cfile215.uf.daum.net/image/994FA6475D4A71B004DC66)

### Task paradigm 
- Participants conducted each of the four sensorimotor tasks including left-hand clenching (LH), right-hand clenching, auditory attention (AD), and visual stimulus (VS) tasks (Jang et al., 2017; Kim et al., 2012).
  - LH / RH : clenched their left or right hand approximately twice per second
  - AD : listened to sounds of 1 kHz during the task period and 900 Hz during the rest period
  - VS : watched an alternating black and white checkerboard with 8 Hz during the task period and watched a black screen during the rest period
- A total of 360 fMRI volumes (i.e., 30 volumes/run × 4 runs/subject × 3 subjects) were available across all tasks and all subjects.



### Blood-oxygenation-level-dependent (BOLD) fMRI data
- 3-T Tim Trio MRI scanner with a 12-channel head coil (Siemens, Erlangen, Germany)
- A standard gradient-echo echo-planar imaging pulse sequence
- TR/TE = 2000/30 ms; flip angle = 90°; in-plane voxels = 64 × 64; 36 axial slices; 4 mm slice thickness without a gap; voxel size = 3.75 × 3.75 × 4.0 mm^3) 



### Preprocessing of BOLD FMRI data
- The fMRI volumes were preprocessed using the SPM8 software toolbox (www.fil.ion.ucl.ac.uk/spm) with standard options
  -  including slice timing correction, motion correction, spatial normalization to the Montreal Neurological Institute space with a 3-mm isotropic voxel size, and spatial smoothing using an 8-mm full-width at half-maximum Gaussian kernel.


  
  *Jang, H., Plis, S. M., Calhoun, V. D., & Lee, J. H. (2017). Task-specific feature extraction and classification of fMRI volumes using a deep neural network initialized with a deep belief network: Evaluation using sensorimotor tasks. Neuroimage, 145, 314-328.*
<bn><bn>
    
*Kim, Y. H., Kim, J., & Lee, J. H. (2012). Iterative approach of dual regression with a sparse prior enhances the performance of independent component analysis for group functional magnetic resonance imaging (fMRI) data. Neuroimage, 63(4), 1864-1889.*
<bn><bn>



---



# Sensorimotor task classification using MLP (Tensorflow) 

## Input data

In [0]:
#!wget http://bspl.korea.ac.kr/sensorimotor.zip
#!unzip sensorimotor.zip


!pip install pydrive
from pydrive.auth import GoogleAuth 
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

data_zip = drive.CreateFile({'id':'145ZZvxYKPjRHegjNzF5gjkI7zkz5gRM7'})
data_zip.GetContentFile('sensorimotor.zip') 
!unzip sensorimotor.zip

"""
1. You will have to choose google account of which it will be accessing google drive.
2. Copy the verification code and paste it into notebook. Press enter after pasting verification code.
ref) https://www.javacodemonk.com/guide-to-read-data-from-google-drive-in-google-colaboratory-079a3609
"""

  
###  Estimation of percentage of BOLD signal
- The BOLD intensities of the fMRI volumes within a task-related period (i.e., three blocks of 20-s task periods after a 6-s delay from the task onset for each fMRI run) were normalized to percent signal change relative to the average BOLD signal with the 80-s baseline period.
- A total of 360 fMRI volumes (i.e., 30 volumes/run × 4 runs/subject × 3 subjects) were available across all tasks and all subjects  
<bn><bn>  

In [0]:
tasks=['LH','RH','AD','VIS']
n_tasks = len(tasks)
n_sbj = 3

# Installation of Nilearn to visualize nii file in Python
!pip install nilearn
from nilearn import image, plotting
from scipy import io as sio

# Load data
image_pct_bold = [None]*n_sbj
task_label = [None]*n_sbj
for sbj in range(n_sbj):
    image_pct_bold[sbj] = image.load_img('sensorimotor_sbj%d.nii'%(sbj+1))
    task_label[sbj] = sio.loadmat('y_sensorimotor_sbj%d.mat'%(sbj+1))['y'].squeeze()
print('\n ==> Data loaded to a variable')

In [0]:
%matplotlib inline
import numpy as np

# Get the dimension of data
[xdim, ydim, zdim, tdim] = image_pct_bold[sbj].shape
tdim_run = int(tdim/n_tasks)

# Examplary subject's average percentage BOLD signal
for run in range(n_tasks): 
    image_plot = image.index_img(image_pct_bold[1], run*tdim_run + np.arange(tdim_run))
    plotting.plot_stat_map(image.mean_img(image_plot), threshold=1, title=tasks[run])
    plotting.show(); 
    ;


### In-brain mask  
- The voxels whose BOLD intensities were greater than 20% of the maximum BOLD intensity for each preprocessed fMRI volume were defined as an in-brain mask from the fMRI volume. 
- Then, the in-brain mask that overlapped across all fMRI volumes and all subjects was used to define the final set of input voxels, resulting in a total of 74,484 in-brain voxels


In [0]:
vMsk_3d = sio.loadmat('vMsk_3d.mat')
vMsk = np.where(vMsk_3d['vMsk_3d'])

###########################################################################
# train_x  = 240 volumes x 74484 voxels
# train_y  = 240 volumes x 4 [one-hot vectors for each of LH, RH, AD and VS]
# test_x  = 120 volumes x 74484 voxels
# test_y  = 120 volumes x 4 [one-hot vectors for each of LH, RH, AD and VS]
###########################################################################

# 2 subjects' vectorized functional data
train_x = np.hstack([image_pct_bold[0].get_data()[vMsk][:],image_pct_bold[1].get_data()[vMsk][:]]).T
# One-hot encoding task labels
train_y = np.hstack([task_label[0],task_label[1]]).T
train_y = np.eye(n_tasks)[train_y]

# 1 subject's vectorized functional data
test_x = image_pct_bold[2].get_data()[vMsk][:].T
# One-hot encoding task labels
test_y = task_label[2]
test_y = np.eye(n_tasks)[test_y]

## Import libraries

In [0]:
# -*- coding: utf-8 -*-

# This import statement gives Python access to all of TensorFlow's classes, methods, and symbols. 
import tensorflow as tf
# The fundamental package for scientific computing with Python.
import numpy as np
# Linear algebra module for calculating L1 and L2 norm  
from numpy import linalg as LA
# To plot the results
import matplotlib.pyplot as plt
# To check the directory when saving the results
import os.path
# The module for file input and output
import scipy.io as sio
# To measure time
import timeit
# To get date information
import datetime
# To get hsp combination
import itertools

##  Define a classifier

![대체 텍스트](http://cfile236.uf.daum.net/image/998625365D4BAA2430FF08)
(Jang et al., 2017)

In [0]:
"""
To create a MLP network  
"""

def init_model(n_nodes, layer_activation):
    
    if layer_activation=='sigmoid':
        layer_activation = tf.nn.sigmoid
    elif layer_activation=='tanh':
        layer_activation = tf.nn.tanh
    elif layer_activation=='relu':
        layer_activation = tf.nn.relu


    # 'node_index' to split placeholder, for example, given hidden_nodes=[100, 100, 100], nodes_index=[0, 100, 200, 300]
    nodes_index = [int(np.sum(n_nodes[1:i+1])) for i in range(np.size(n_nodes,axis=0)-1)]

    # Make two placeholders to fill the values later when training or testing
    X = tf.placeholder(tf.float32,[None,n_nodes[0]])
    Y = tf.placeholder(tf.float32,[None,n_nodes[-1]])

    # Create randomly initialized weight variables
    w_init = [tf.math.divide(tf.random_normal([n_nodes[i],n_nodes[i+1]]), tf.sqrt(float(n_nodes[i])/2)) for i in range(np.size(n_nodes,axis=0)-1)]
    w = [tf.Variable(w_init[i], dtype=tf.float32) for i in range(np.size(n_nodes,axis=0)-1)]
    # Create zero initialized bias variables
    b = [tf.Variable(tf.zeros([n_nodes[i+1]]), dtype=tf.float32) for i in range(np.size(n_nodes,axis=0)-1)]

    # Build an MLP model
    mlp_layers=[0.0]*(np.size(n_nodes,axis=0)-1)
    for i in range(np.size(n_nodes,axis=0)-1):
        # Input layer
        if i==0:
            mlp_layers[i] = tf.add(tf.matmul(X,w[i]),b[i])
            mlp_layers[i] = layer_activation(mlp_layers[i])
        # Hidden layers
        elif i>0 and i<(np.size(n_nodes,axis=0)-1):
            mlp_layers[i] = tf.add(tf.matmul(mlp_layers[i-1],w[i]),b[i])
            mlp_layers[i] = layer_activation(mlp_layers[i])
        # Output layer
        elif i==(np.size(n_nodes,axis=0)-1):
            mlp_layers[i] = tf.add(tf.matmul(mlp_layers[i-1],w[i]),b[i])


    return nodes_index, X, Y, mlp_layers, w, b

## Define training parameters

In [0]:
"""
L1 and L2 regularization terms

""" 

def init_regularizations(mode, n_nodes, nodes_index, n_tg_hsp, w, b):

    if mode=='layer':
        # The size is same with the number of layers
        Beta = tf.placeholder(tf.float32,[n_tg_hsp])
        # Get L1 loss term by simply multiplying beta(scalar value) and L1 norm of weight for each layer
        L1_loss = [tf.reduce_sum(tf.multiply(tf.abs(w[i]),Beta[i])) for i in range(n_tg_hsp)]

    elif mode=='node':
        # The size is same with the number of nodes
        Beta = tf.placeholder(tf.float32,[np.sum(n_nodes[1:(n_tg_hsp+1)])])
        # Get L1 loss term by multiplying beta(vector values as many as n_nodes) and L1 norm of weight for each layer
        L1_loss = [tf.reduce_sum(tf.matmul(tf.abs(w[i]), tf.reshape(Beta[nodes_index[i]:nodes_index[i+1]],[-1,1]))) for i in range(n_tg_hsp)] 
        
    L1_loss_total = tf.reduce_sum(L1_loss)

    L2_REG = tf.placeholder(tf.float32)
    L2_loss = [tf.reduce_sum(tf.square(w[i])) for i in range(np.size(n_nodes,axis=0)-1)]
    L2_loss_total = tf.multiply(L2_REG,tf.reduce_sum(L2_loss))

    return Beta, L1_loss_total, L2_REG, L2_loss_total


""" 
Cost term (Cost = cross entropy + L1 term + L2 term ) 
"""
def init_cost(mlp_layers, Y, L1_loss_total, L2_loss_total):

    # A softmax regression : it adds up the evidence of our input being in certain classes, and converts that evidence into probabilities.
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=mlp_layers[-1], labels=Y)) \
                             + L1_loss_total + L2_loss_total

    return cost


""" 
Error term  
"""
def init_error(mlp_layers, Y):

    correct_prediction = tf.equal(tf.argmax(mlp_layers[-1],1),tf.argmax(Y,1))
    error = 1-tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

    
    return error


"""
TensorFlow provides optimizers that slowly change each variable in order to minimize the loss function.
"""
def init_optimizer(optimizer_algorithm, cost, momentum):
    
    Lr = tf.placeholder(tf.float32)
    
    if optimizer_algorithm=='GradientDescent':
        optimizer = tf.train.GradientDescentOptimizer(Lr).minimize(cost)
    elif optimizer_algorithm=='Adagrad':
        optimizer = tf.train.AdagradOptimizer(Lr).minimize(cost)
    elif optimizer_algorithm=='Adam':
        optimizer = tf.train.AdamOptimizer(Lr).minimize(cost)
    elif optimizer_algorithm=='Momentum':
        optimizer = tf.train.MomentumOptimizer(Lr,momentum).minimize(cost)
    elif optimizer_algorithm=='RMSProp':
        optimizer = tf.train.RMSPropOptimizer(Lr).minimize(cost)

    return Lr, optimizer



"""
Variables to store training results
"""
def init_savingVariables(mode, n_tg_hsp):
    if mode=='layer':
        beta_val = np.zeros(n_tg_hsp)
        beta_vec = np.zeros(n_tg_hsp)
        hsp_val = np.zeros(n_tg_hsp)
        result_beta = np.zeros(n_tg_hsp)
        result_hsp = np.zeros(n_tg_hsp)

    elif mode=='node':
        beta_val = [np.zeros(n_nodes[i+1]) for i in range(n_tg_hsp)]
        beta_vec = np.zeros(np.sum(n_nodes[1:-1]))
        hsp_val = [np.zeros(n_nodes[i+1]) for i in range(n_tg_hsp)]
        result_beta = [np.zeros(n_nodes[i+1]) for i in range(n_tg_hsp)]
        result_hsp = [np.zeros(n_nodes[i+1]) for i in range(n_tg_hsp)]


    # make arrays to store and plot results
    result_lr = np.zeros(1)
    result_cost = np.zeros(1)
    result_train_err = np.zeros(1)
    result_test_err = np.zeros(1)

    return beta_val, beta_vec, hsp_val, result_beta, result_hsp, result_lr, result_cost, result_train_err, result_test_err


## Define Hoyer's sparsity control function

![대체 텍스트](http://cfile211.uf.daum.net/image/99936D3C5D4BA942271035)

In [0]:
"""
Hoyer's sparsity level control
"""

def Hoyers_sparsity_control(mode, beta_lrates, W, b, max_b, tg):

    # Weight sparsity control with Hoyer's sparsness (Layer wise)
    if mode=='layer':

        # Get value of weight
        [n_nodes,dim]=W.shape
        num_elements=n_nodes*dim

        Wvec=W.flatten()

        # Calculate L1 and L2 norm
        L1=LA.norm(Wvec,1)
        L2=LA.norm(Wvec,2)

        # Calculate hoyer's sparsness
        h=(np.sqrt(num_elements)-(L1/L2))/(np.sqrt(num_elements)-1)

        # Update beta
        b-=beta_lrates*np.sign(h-tg)

        # Trim value
        b=0.0 if b<0.0 else b
        b=max_b if b>max_b else b

        return [h,b]

    # Weight sparsity control with Hoyer's sparsness (Node wise)
    elif mode=='node':

        b_vec = b

        # Get value of weight
        [n_nodes,dim]=W.shape

        # Calculate L1 and L2 norm
        L1=LA.norm(W,1,axis=0)
        L2=LA.norm(W,2,axis=0)

        h_vec = np.zeros((1,dim))
        tg_vec = np.ones(dim)*tg

        # Calculate hoyer's sparsness
        h_vec=(np.sqrt(n_nodes)-(L1/L2))/(np.sqrt(n_nodes)-1)

        # Update beta
        b_vec-=beta_lrates*np.sign(h_vec-tg_vec)

        # Trim value
        b_vec[b_vec<0.0]=0.0
        b_vec[b_vec>max_b]=max_b


        return [h_vec,b_vec]



## Set hyperparameters for training

In [0]:
""" 
Set the number of nodes for input, hidden layers and output layer
"""
n_nodes=[74484,100,100,100,4]



"""
Select optimizer :
    'GradientDescent'
    'Adagrad'
    'Adam'
    'Momentum'
    'RMSProp'
"""
optimizer_algorithm = 'Momentum'

momentum = 0.5


"""
Set learning hyperparameters
"""

# Total epochs
n_epochs = 250
# Mini batch size
batch_size = 30
# Initial learning rate
lr_init = 0.002
# Learning rate anealing after **th epoch
begin_anneal = 100
# Learning rate decay rate
decay_rate = 0.001
# Minimum learning rate
lr_min = 0.0002

# Layer activation 'sigmoid', 'tanh', 'relu'
layer_activation = 'tanh'

# Lambda for L2 regularization
L2_reg = 1e-4


"""
Select the sparsity control mode
'layer' for layer wise sparsity control
'node' for node wise sparsity control
"""

mode = 'layer'

# Max beta
max_beta = [0.01, 0.2, 0.2] #[0.02, 0.5, 0.5]
# Beta learning rate
beta_lr = [0.002, 0.05, 0.05]

# Candidates of target sparsness level for each layer (0:dense~1:sparse)
hsp_cands = list(itertools.product([0.8, 0.6],[0.8, 0.6],[0.8, 0.6])) #[[0.8, 0.8, 0.8]]
n_hsp_cands = len(hsp_cands)


"""
Set the directory to save results
"""
directory_path = os.getcwd() #'/users/hailey/code/03_code/Tensorflow_tutorial/dnnwsp-master/Tensorflow_code'

# make a new 'results' directory in the current directory
dtime = datetime.datetime.now()
directory_save = os.path.join(directory_path, r'results_%04d%02d%02d_%02d%02d'%(dtime.year,dtime.month,dtime.day,dtime.hour,dtime.minute))
if not os.path.exists(directory_save):
    os.makedirs(directory_save)

## Create a model using definitions above

In [0]:
nodes_index, X, Y, mlp_layers, w, b = init_model(n_nodes, layer_activation)

Beta, L1_loss_total, L2_REG, L2_loss_total = init_regularizations(mode, n_nodes, nodes_index, len(hsp_cands[0]), w, b)
cost = init_cost(mlp_layers, Y, L1_loss_total, L2_loss_total)
error = init_error(mlp_layers, Y)

Lr, optimizer = init_optimizer(optimizer_algorithm, cost, momentum)

## Learning

In [0]:
start_time = timeit.default_timer()

error_vd = [None]*n_hsp_cands
hsp_vd = [None]*n_hsp_cands

for i_tg in range(n_hsp_cands+1):
    
        
    ### In case of a validation phase 
    if i_tg < n_hsp_cands:      
        vd_ts = 'Validation'
        # One of candidates
        hsp_tg = hsp_cands[i_tg]
        n_hsp_tg = len(hsp_tg)
        
        # One subject from train data as inner training set
        train_x_set = train_x[:int(np.size(train_x,axis=0)/2)][:]
        train_y_set = train_y[:int(np.size(train_y,axis=0)/2)][:]
        # The other subject from train data as inner validation set
        test_x_set = train_x[int(np.size(train_x,axis=0)/2):][:]
        test_y_set = train_y[int(np.size(train_y,axis=0)/2):][:]
        print("\n ****************************** Validation phase *******************************")
        print("# (",i_tg+1,"/",n_hsp_cands,") hsp candidate :",hsp_tg,"#")
        
    ### In case of a test phase
    else:
        vd_ts = 'Test'
        # To set a target as an optimal sparsity level
        i_selected = np.argmin(error_vd)      
        hsp_tg = hsp_cands[i_selected]
        n_hsp_tg = len(hsp_tg) 

        train_x_set = train_x[:][:]
        train_y_set = train_y[:][:]
        test_x_set = test_x[:][:]
        test_y_set = test_y[:][:]
        print("\n ****************************** Test phase *******************************")
        print("############### ==> Selected hsp :",hsp_tg,"###############")


        
        
    # Training starts    
    with tf.Session(config=tf.ConfigProto(log_device_placement=False)) as sess:
        
        beta_val, beta_vec, hsp_val, result_beta, result_hsp, result_lr, result_cost, result_train_err, result_test_err = init_savingVariables(mode, n_hsp_tg)

        
        # To initialize all the variables in a TensorFlow program, you must explicitly call a special operation
        init = tf.global_variables_initializer()
        sess.run(init)


        # Start training
        for epoch in range(n_epochs):

            # Shuffle training data at the begining of each epoch
            n_samples = np.size(train_x_set, axis=0)
            shuff_ids = np.arange(n_samples)
            np.random.shuffle(shuff_ids)


            # Begin annealing
            if epoch == 0:
                lr = lr_init
            elif decay_rate!=0 and (epoch+1)>begin_anneal:
                lr = max( lr_min, (-decay_rate*(epoch+1) + (1+decay_rate*begin_anneal)) * lr )


            
            cost_epoch=0.0
            
            # How many mini-batch iterations we need
            n_batches = int(np.ceil(n_samples/batch_size))
            
            # minibatch based training
            for batch in range(n_batches):
                first_sample = batch*batch_size
                last_sample = min((batch+1)*batch_size, n_samples)
                batch_x = train_x_set[shuff_ids[first_sample:last_sample]]
                batch_y = train_y_set[shuff_ids[first_sample:last_sample]]
  
                # Get cost and optimize the model
                cost_batch,_ = sess.run([cost,optimizer],{Lr:lr, X:batch_x, Y:batch_y, Beta:beta_vec, L2_REG:L2_reg})
                cost_epoch += cost_batch/n_batches

                # Weight sparsity control
                if mode=='layer':
                    for i in range(n_hsp_tg):
                        [hsp_val[i], beta_val[i]] = Hoyers_sparsity_control(mode, beta_lr[i], sess.run(w[i]), beta_val[i], max_beta[i], hsp_tg[i])
                    beta_vec = beta_val[:]

                elif mode=='node':
                    for i in range(n_hsp_tg):
                        [hsp_val[i], beta_val[i]] = Hoyers_sparsity_control(mode, beta_lr[i], sess.run(w[i]), beta_val[i], max_beta[i], hsp_tg[i])
                    # flatten beta_val (shape (3, 100) -> (300,))
                    beta_vec = [item for sublist in beta_val for item in sublist]

            # get train and test error
            train_err_epoch = sess.run(error,{X:train_x_set, Y:train_y_set}) 
            test_err_epoch = sess.run(error,{X:test_x_set, Y:test_y_set})
            

            # Save the results to array
            result_train_err = np.hstack([result_train_err,[train_err_epoch]])
            result_test_err = np.hstack([result_test_err,[test_err_epoch]])
            result_lr = np.hstack([result_lr,[lr]])
            result_cost = np.hstack([result_cost,[cost_epoch]])

            if mode=='layer':
                result_hsp=[np.vstack([result_hsp[i],[hsp_val[i]]]) for i in range(n_hsp_tg)]
                result_beta=[np.vstack([result_beta[i],[beta_val[i]]]) for i in range(n_hsp_tg)]

            elif mode=='node':
                result_hsp=[np.vstack([result_hsp[i],[np.transpose(hsp_val[i])]]) for i in range(n_hsp_tg)]
                result_beta=[np.vstack([result_beta[i],[np.transpose(beta_val[i])]]) for i in range(n_hsp_tg)]

            # Print result at epoch 
            if ((epoch+1)%50==0):             
                print("< Epoch", "{:02d}".format(epoch+1),"> Cost :", "{:.4f}".format(cost_epoch)\
                                                ,"/ Train err :", "{:.4f}".format(train_err_epoch),"/ ",vd_ts," err :","{:.4f}".format(test_err_epoch))
                print("             beta :",np.mean(np.swapaxes(result_beta,0,1)[-1],axis=1),"/ hsp :",np.mean(np.swapaxes(result_hsp,0,1)[-1],axis=1))

                
        print("")
        print("-> Accuracy :", "{:.3f}".format(1-result_test_err[-1]))
        
        
            
        # Plot and save the figures
        
        # Make a new results directory for the target
        hsp_tg_str=['0.%02d-'%(i*100) for i in hsp_tg]
        hsp_tg_str[-1]=hsp_tg_str[-1][:-1]
        hsp_tg_str=''.join(hsp_tg_str)

        directory_results = os.path.join(directory_save, r'%s_hsp_tg_%s'%(vd_ts,hsp_tg_str))
        if not os.path.exists(directory_results):
            os.makedirs(directory_results)


        # Plot the change of learning rate
        plt.figure()
        plt.title("Learning rate plot",fontsize=16)
        result_lr=result_lr[1:]
        plt.ylim(0.0, lr_init*1.2)
        plt.plot(result_lr)
        plt.savefig(directory_results+'/learning_rate.png')
        plt.show(block=False)


        # Plot the change of cost
        plt.figure()
        plt.title("Cost plot",fontsize=16)
        result_cost=result_cost[1:]
        plt.plot(result_cost)
        plt.savefig(directory_results+'/cost.png')
        plt.show(block=False)


        # Plot train & test error
        plt.figure()
        plt.title("Train & Test error plot",fontsize=16)
        result_train_err=result_train_err[1:]
        plt.plot(result_train_err, label='Train error')
        result_test_err=result_test_err[1:]
        plt.plot(result_test_err, label='%s error'%vd_ts)
        plt.ylim(0.0, 1.0)
        plt.legend(loc='upper right')
        plt.show(block=False)
        plt.savefig(directory_results+'/error_'+"tr{:.4f}_ts{:.4f}".format(result_train_err[-1],result_test_err[-1])+'.png')


        # Plot the change of beta value
        print("")
        plt.figure()
        for i in range(n_hsp_tg):
            print("")
            plt.title("Beta plot",fontsize=16)
            result_beta[i]=result_beta[i][1:]
            plt.plot(result_beta[i], label='layer%d'%(i+1) if mode=='layer' else '')
            plt.ylim(0.0, np.max(max_beta)*1.2)
        plt.legend(loc='best')
        plt.show(block=False)
        plt.savefig(directory_results+'/beta.png')


        # Plot the change of Hoyer's sparsity level
        print("")
        plt.figure()
        for i in range(n_hsp_tg):
            print("")
            plt.title("Hoyer's sparsity plot",fontsize=16)
            result_hsp[i]=result_hsp[i][1:]
            plt.plot(result_hsp[i], label='layer%d'%(i+1) if mode=='layer' else '')
            plt.ylim(0.0, 1.0)
        plt.legend(loc='best')
        plt.show(block=False)
        if n_hsp_tg == 3:
            plt.savefig(directory_results + '/hsp_final_' + "{:.3f}".format(
                np.mean(result_hsp[0][-1])) + '_' + "{:.3f}".format(
                np.mean(result_hsp[1][-1])) + '_' + "{:.3f}".format(
                np.mean(result_hsp[2][-1])) + '.png')
        elif n_hsp_tg == 2:
            plt.savefig(directory_results + '/hsp_final_' + "{:.3f}".format(
                np.mean(result_hsp[0][-1])) + '_' + "{:.3f}".format(
                np.mean(result_hsp[1][-1])) + '.png')
        elif n_hsp_tg == 1:
            plt.savefig(directory_results + '/hsp_final_' + "{:.3f}".format(
                np.mean(result_hsp[0][-1])) + '.png')
            
            
        # save results as .mat file
        sio.savemat(directory_results+"/result_learningrate.mat", mdict={'lr': result_lr})
        sio.savemat(directory_results+"/result_cost.mat", mdict={'cost': result_cost})
        sio.savemat(directory_results+"/result_train_err.mat", mdict={'trainErr': result_train_err})
        sio.savemat(directory_results+"/result_test_err.mat", mdict={'testErr': result_test_err})
        sio.savemat(directory_results+"/result_beta.mat", mdict={'beta': result_beta})
        sio.savemat(directory_results+"/result_hsp.mat", mdict={'hsp': result_hsp})
        sio.savemat(directory_results+"/result_weight.mat", mdict={'weight': sess.run(w)})
        sio.savemat(directory_results+"/result_bias.mat", mdict={'bias': sess.run(b)})
        
        # If it was validation phase
        if i_tg < n_hsp_cands: 
            plt.close('all')
            error_vd[i_tg] = result_test_err[-1]
            hsp_vd[i_tg] = np.mean(np.swapaxes(result_hsp,0,1)[-1],axis=1)
            
            
        
end_time = timeit.default_timer()

print('\nThe code ran for %d mins' %((end_time-start_time)/60)+'\n\n')

## Save parameters and summary as text file

In [0]:
f = open(directory_save+"/parameters_summary.txt",'w') 
f.write('mode : '+str(mode)+'\n')
f.write('optimizer_algorithm : '+str(optimizer_algorithm)+'\n')
f.write('layer_activation : '+str(layer_activation)+'\n')
f.write('n_epochs : '+str(n_epochs)+'\n')
f.write('batch_size : '+str(batch_size)+'\n')
f.write('lr_init : '+str(lr_init)+'\n')
f.write('lr_min : '+str(lr_min)+'\n')
f.write('begin_anneal : '+str(begin_anneal)+'\n')
f.write('decay_rate : '+str(decay_rate)+'\n')
f.write('L2_reg : '+str(L2_reg)+'\n')
f.write('max_beta : '+str(max_beta)+'\n')
f.write('beta_lr : '+str(beta_lr)+'\n')
f.write('---\n')
for i_tg in range(n_hsp_cands):
    f.write('('+str(i_tg+1)+'/'+str(n_hsp_cands)+') tg :'+str(hsp_cands[i_tg])+'/ hsp : '+str(hsp_vd[i_tg])+'- val error : '+"{:.4f}".format(error_vd[i_tg])+'\n')
f.write('==> Selected tg :'+str(hsp_cands[i_selected])+'/ hsp : '+str(np.mean(np.swapaxes(result_hsp,0,1)[-1],axis=1))+'- ts error : '+"{:.4f}".format(result_test_err[-1])+'\n')
f.write('\n The code ran for %d mins' %((end_time-start_time)/60)+'\n\n')
f.close()
